необходимо добавить в ноутбук датасет https://kaggle.com/datasets/3f172da82a4d446227bd7ee07ad45233a4f264d44f2def5cdd52239f8ebedef8

In [ ]:
!pip install jiwer==3.0.1
!pip install datasets==2.10.1

In [4]:
import os
import pandas as pd
import json
import shutil
from datasets import load_dataset

In [9]:
def get_full_path(x):
    return f"/kaggle/input/golos-crowd-test/test/crowd/{x['full_path']}"

In [13]:
with open('/kaggle/input/golos-crowd-test/test/crowd/manifest.jsonl', 'r') as json_file:
    json_list = list(json_file)

In [17]:
df_golos = pd.DataFrame.from_dict(list(map(json.loads, json_list)))
df_golos['file_name'] = df_golos['id'].apply(lambda x: f'test/{x}.wav')
df_golos['source'] = f'golos_crowd_test' 
df_golos = df_golos.rename(columns={"text": "answer", "audio_filepath": "full_path"})
df_golos['full_path'] = df_golos.apply(lambda x: get_full_path(x), axis=1)
df_golos.drop(columns=['id'], inplace=True)
df_golos.reset_index(inplace=True, drop=True)
df_golos

,full_path,answer,duration,file_name,source
0,/kaggle/input/golos-crowd-test/test/crowd/file...,афина воспроизведи музыку вперемешку,4.900000,test/e632f7d39c15e7edfc665b91e6f2071f.wav,golos_crowd_test
1,/kaggle/input/golos-crowd-test/test/crowd/file...,найти сериал григорий р,3.652000,test/5db5df8bb9e3b6660b2a04b34d4a355d.wav,golos_crowd_test
2,/kaggle/input/golos-crowd-test/test/crowd/file...,прямой эфир апл манчестер юнайтед тоттенхэм,4.341750,test/2c471aedc6979109f28cd53c58f8c4fb.wav,golos_crowd_test
3,/kaggle/input/golos-crowd-test/test/crowd/file...,ильвиром ивановичем ворончихиным,3.900000,test/756a137ee9debde4a008adc4a4121dc7.wav,golos_crowd_test
4,/kaggle/input/golos-crowd-test/test/crowd/file...,можешь показать киношку исходный код,3.320000,test/1ee3b00170123a6723a40e129b2f6bce.wav,golos_crowd_test
...,...,...,...,...,...
9989,/kaggle/input/golos-crowd-test/test/crowd/file...,рудню,2.260000,test/73bb9272fe3724eb6212e33186f63152.wav,golos_crowd_test
9990,/kaggle/input/golos-crowd-test/test/crowd/file...,найти мульт щенячий патруль,3.020000,test/c872c6f50709a2696b0927725dc86ded.wav,golos_crowd_test
9991,/kaggle/input/golos-crowd-test/test/crowd/file...,стол из эпоксидной смолы,3.598688,test/03b92263995462b2e09655a8e29d4d0b.wav,golos_crowd_test
9992,/kaggle/input/golos-crowd-test/test/crowd/file...,шесть четыреста семьдесят шесть четыреста двад...,11.795312,test/09ac1f9718c6da18380b4a01d92d2561.wav,golos_crowd_test


In [18]:
df_golos['duration'].sum()/3600

11.200331631944444

In [19]:
# создаем папки, где создадим датасет нужный структуры для загрузки на HuggingFace

os.mkdir("/kaggle/working/golos")
os.mkdir("/kaggle/working/golos/test")

In [20]:
# перемещаем туда выбранные файлы и датафрейм с метаданными 

for index, row in df_golos.iterrows():
    if index % 10000 == 0:
        print(index)
    try:
        shutil.copy(row['full_path'], f"/kaggle/working/golos/{row['file_name']}")
    except:
        print('except', index)
        df_golos.drop(index, inplace=True)
        


df_golos.drop(columns=['full_path'], inplace=True)
df_golos.to_csv('/kaggle/working/golos/metadata.csv')

0


In [21]:
# создаем датасет локально

dataset = load_dataset("audiofolder", data_dir='/kaggle/working/golos')

Resolving data files:   0%|          | 0/9994 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/default-70fde7b8c2ab0b7a/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
# загружаем на HuggingFace

from huggingface_hub import login
login(token='') # добавить write токен нужного аккаунта HuggingFace 

DATASET_NAME = 'aanosov/golos_test'

dataset.push_to_hub(DATASET_NAME, private=True)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


Map:   0%|          | 0/3332 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/3331 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]